In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import torch
import torchtuples as tt

from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from pycox.models import DeepHit
from pycox.evaluation import EvalSurv



np.random.seed(1234)
_ = torch.manual_seed(1234)


df_train = pd.read_excel("Melanoma_train_data_py.xls")
df_test = pd.read_excel("Melanoma_test_data_py.xls")

## Dealing with Missing values

#df_train['meal.cal'].fillna(df_train['meal.cal'].mean(), inplace=True)
#df_train['wt.loss'].fillna(df_train['wt.loss'].mean(), inplace=True)
#df_test['meal.cal'].fillna(df_test['meal.cal'].mean(), inplace=True)
#df_test['wt.loss'].fillna(df_test['wt.loss'].mean(), inplace=True)


df_train = df_train.dropna()
df_test = df_test.dropna()


get_x = lambda df: (df.drop(columns=['time', 'status']).values.astype('float32'))
x_train = get_x(df_train)
x_test = get_x(df_test)

In [2]:
class LabTransform(LabTransDiscreteTime):
    def transform(self, durations, events):
        durations, is_event = super().transform(durations, events > 0)
        events[is_event == 0] = 0
        return durations, events.astype('int64')

num_durations = 108
labtrans = LabTransform(num_durations)
get_target = lambda df: (df['time'].values, df['status'].values)


y_train = labtrans.fit_transform(*get_target(df_train))
y_test = labtrans.fit_transform(*get_target(df_test))
durations_test, events_test = get_target(df_test)

In [3]:
class SimpleMLP(torch.nn.Module):
    """Simple network structure for competing risks.
    """
    def __init__(self, in_features, num_nodes, num_risks, out_features, batch_norm=True,
                 dropout=None):
        super().__init__()
        self.num_risks = num_risks
        self.mlp = tt.practical.MLPVanilla(
            in_features, num_nodes, num_risks * out_features,
            batch_norm, dropout,
        )
        
    def forward(self, input):
        out = self.mlp(input)
        return out.view(out.size(0), self.num_risks, -1)


in_features = x_train.shape[1]
num_risks = y_train[1].max()
out_features = len(labtrans.cuts)
dropout = [0.0]



list_num_nodes_shared = [[32, 8], [16, 8], [16, 4], [8, 4]]
list_batch_norm = [False]
list_lr = [0.01, 0.001, 0.001]
list_alpha = [0.1, 0.2, 0.3, 0.4]
list_sigma = [0.1, 0.2, 0.3, 0.4]
list_batch_size = [128, 256]


In [4]:
parameters = []
for num_nodes_shared in list_num_nodes_shared:
    for batch_norm in list_batch_norm:
        for lr in list_lr:
            for alpha in list_alpha:
                for sigma in list_sigma:
                    for batch_size in list_batch_size:
                        parameters.append([num_nodes_shared, batch_norm, lr, alpha, sigma, batch_size])


smlp_cv_results = pd.DataFrame(parameters)
smlp_cv_results["cindex"] = 0


kf = KFold(n_splits = 5)



for index in range(smlp_cv_results.shape[0]):
    num_nodes_shared = smlp_cv_results.iloc[index, 0]
    batch_norm = smlp_cv_results.iloc[index, 1]
    lr = smlp_cv_results.iloc[index, 2]
    alpha = smlp_cv_results.iloc[index, 3]
    sigma = smlp_cv_results.iloc[index, 4]
    batch_size = smlp_cv_results.iloc[index, 5]
    
    cindexes = []
    for train_index, test_index in kf.split(df_train):
        X_tr = x_train[train_index, ]
        X_val = x_train[test_index, ]
        Y_tr_0 = y_train[0][train_index, ]
        Y_tr_1 = y_train[1][train_index, ]
        Y_val_0 = y_train[0][test_index, ]
        Y_val_1 = y_train[1][test_index, ]
        Y_tr = (Y_tr_0, Y_tr_1)
        Y_val = (Y_val_0, Y_val_1)
        
        net = SimpleMLP(in_features, num_nodes_shared, num_risks, out_features, batch_norm)
        optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01,
                            cycle_eta_multiplier=0.8)
        model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma,
                        duration_index = labtrans.cuts)

        epochs = 100
        callbacks = [tt.callbacks.EarlyStoppingCycle()]
        verbose = False # set to True if you want printout

        log = model.fit(X_tr, Y_tr, int(batch_size), epochs,
                callbacks, verbose, val_data = (X_val, Y_val))
        
        cif = model.predict_cif(x_test)
        cif1 = pd.DataFrame(cif[0], model.duration_index)
        ev1 = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')
        c_index = ev1.concordance_td()

        cindexes.append(c_index)


    smlp_cv_results.iloc[index, 6] = np.mean(cindexes)
    smlp_cv_results.to_csv('cv.results.smlp.csv', index = False)
    print(smlp_cv_results.iloc[index, ].values)

C:\Users\USR\anaconda3\lib\site-packages\torchtuples\callbacks.py:607: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1174.)
  p.data = p.data.add(-weight_decay * eta, p.data)


[list([32, 8]) False 0.01 0.1 0.1 128 0.583240997229917]
[list([32, 8]) False 0.01 0.1 0.1 256 0.46509695290858727]
[list([32, 8]) False 0.01 0.1 0.2 128 0.5777008310249307]
[list([32, 8]) False 0.01 0.1 0.2 256 0.6138504155124654]
[list([32, 8]) False 0.01 0.1 0.3 128 0.6]
[list([32, 8]) False 0.01 0.1 0.3 256 0.5346260387811634]
[list([32, 8]) False 0.01 0.1 0.4 128 0.48310249307479225]
[list([32, 8]) False 0.01 0.1 0.4 256 0.5886426592797784]
[list([32, 8]) False 0.01 0.2 0.1 128 0.43337950138504155]
[list([32, 8]) False 0.01 0.2 0.1 256 0.49529085872576173]
[list([32, 8]) False 0.01 0.2 0.2 128 0.48171745152354567]
[list([32, 8]) False 0.01 0.2 0.2 256 0.5519390581717452]
[list([32, 8]) False 0.01 0.2 0.3 128 0.503601108033241]
[list([32, 8]) False 0.01 0.2 0.3 256 0.5501385041551247]
[list([32, 8]) False 0.01 0.2 0.4 128 0.5185595567867036]
[list([32, 8]) False 0.01 0.2 0.4 256 0.5001385041551247]
[list([32, 8]) False 0.01 0.3 0.1 128 0.4617728531855955]
[list([32, 8]) False 0.01 

[list([16, 8]) False 0.001 0.2 0.3 128 0.4382271468144044]
[list([16, 8]) False 0.001 0.2 0.3 256 0.4293628808864266]
[list([16, 8]) False 0.001 0.2 0.4 128 0.38088642659279776]
[list([16, 8]) False 0.001 0.2 0.4 256 0.4975069252077562]
[list([16, 8]) False 0.001 0.3 0.1 128 0.3911357340720222]
[list([16, 8]) False 0.001 0.3 0.1 256 0.46662049861495847]
[list([16, 8]) False 0.001 0.3 0.2 128 0.4422437673130194]
[list([16, 8]) False 0.001 0.3 0.2 256 0.4671745152354571]
[list([16, 8]) False 0.001 0.3 0.3 128 0.443628808864266]
[list([16, 8]) False 0.001 0.3 0.3 256 0.41925207756232685]
[list([16, 8]) False 0.001 0.3 0.4 128 0.43656509695290857]
[list([16, 8]) False 0.001 0.3 0.4 256 0.4667590027700831]
[list([16, 8]) False 0.001 0.4 0.1 128 0.4475069252077562]
[list([16, 8]) False 0.001 0.4 0.1 256 0.42049861495844876]
[list([16, 8]) False 0.001 0.4 0.2 128 0.4786703601108033]
[list([16, 8]) False 0.001 0.4 0.2 256 0.38227146814404434]
[list([16, 8]) False 0.001 0.4 0.3 128 0.4313019390

[list([16, 4]) False 0.001 0.3 0.4 256 0.4807479224376731]
[list([16, 4]) False 0.001 0.4 0.1 128 0.4918282548476454]
[list([16, 4]) False 0.001 0.4 0.1 256 0.4426592797783934]
[list([16, 4]) False 0.001 0.4 0.2 128 0.4958448753462604]
[list([16, 4]) False 0.001 0.4 0.2 256 0.5259002770083102]
[list([16, 4]) False 0.001 0.4 0.3 128 0.4473684210526316]
[list([16, 4]) False 0.001 0.4 0.3 256 0.5142659279778392]
[list([16, 4]) False 0.001 0.4 0.4 128 0.5452908587257618]
[list([16, 4]) False 0.001 0.4 0.4 256 0.41828254847645424]
[list([8, 4]) False 0.01 0.1 0.1 128 0.4768698060941829]
[list([8, 4]) False 0.01 0.1 0.1 256 0.4934903047091413]
[list([8, 4]) False 0.01 0.1 0.2 128 0.5441828254847646]
[list([8, 4]) False 0.01 0.1 0.2 256 0.5519390581717452]
[list([8, 4]) False 0.01 0.1 0.3 128 0.52797783933518]
[list([8, 4]) False 0.01 0.1 0.3 256 0.554432132963989]
[list([8, 4]) False 0.01 0.1 0.4 128 0.5229916897506925]
[list([8, 4]) False 0.01 0.1 0.4 256 0.516759002770083]
[list([8, 4]) Fa

In [6]:
smlp_cv_results = pd.read_csv("cv.results.smlp.csv")
print(smlp_cv_results["cindex"].values.max())
ind_best = smlp_cv_results["cindex"].values.argmax()

num_nodes_shared = eval(smlp_cv_results.iloc[ind_best, 0])
batch_norm = smlp_cv_results.iloc[ind_best, 1]
lr = smlp_cv_results.iloc[ind_best, 2]
alpha = smlp_cv_results.iloc[ind_best, 3]
sigma = smlp_cv_results.iloc[ind_best, 4]
batch_size = smlp_cv_results.iloc[ind_best, 5]


net = SimpleMLP(in_features, num_nodes_shared, num_risks, out_features, batch_norm)

optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01, 
                            cycle_eta_multiplier=0.8)
model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma, duration_index = labtrans.cuts)

epochs = 100
callbacks = [tt.callbacks.EarlyStopping(patience = 3)]
verbose = True # set to True if you want printout

log = model.fit(x_train, y_train, int(batch_size), epochs, callbacks, verbose, val_data = (x_test, y_test))
        
cif = model.predict_cif(x_test)
cif1 = pd.DataFrame(cif[0], model.duration_index)
ev = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')
c_index = ev.concordance_td()
    
ibs = ev.integrated_brier_score(np.linspace(0, durations_test.max(), 108))


print('C-index: {:.4f}'.format(c_index))
print('IBS: {:.4f}'.format(ibs))




def bootstrap_replicate_1d(data):
    bs_sample = np.random.choice(data,len(data))
    return bs_sample


bootstrap_R = 100
c_indexes = []
ibss = []


for i in range(bootstrap_R):
    print(i)
    train_bs_idx = bootstrap_replicate_1d(np.array(range(df_train.shape[0])))
    # Creating the X, T and E input
    X_train = x_train[train_bs_idx, ]
    T_train = y_train[0][train_bs_idx]
    E_train = y_train[1][train_bs_idx]
    Y_train = (T_train, E_train)
    
    net = SimpleMLP(in_features, num_nodes_shared, num_risks, out_features, batch_norm)
    optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01, cycle_eta_multiplier=0.8)
    model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma, duration_index = labtrans.cuts)
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience = 3)]
    log = model.fit(x_train, y_train, int(batch_size), epochs, callbacks, verbose, val_data = (x_test, y_test))
    cif = model.predict_cif(x_test)
    cif1 = pd.DataFrame(cif[0], model.duration_index)
    ev = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')
    c_index = ev.concordance_td()
    ibs = ev.integrated_brier_score(np.linspace(0, durations_test.max(), 108))
    c_indexes.append(np.round(c_index, 4))
    ibss.append(np.round(ibs, 4))


pd.DataFrame(data = {"cindex": c_indexes, "ibs": ibss}).to_csv("results.ci.smlp.csv", index=False)

# Compute the 95% confidence interval: conf_int
mean_cindex = np.mean(c_indexes)
mean_ibs = np.mean(ibss)


# Print the mean
print('mean cindex =', mean_cindex)
print('mean ibs =', mean_ibs)


ci_cindex = np.percentile(c_indexes, [2.5, 97.5])
ci_ibs = np.percentile(ibss, [2.5, 97.5])

# Print the confidence interval
print('ci_cindex confidence interval =', ci_cindex)
print('ci_ibs confidence interval =', ci_ibs)

0.6162049861495845
0:	[0s / 0s],		train_loss: 2.1901,	val_loss: 1.4441
1:	[0s / 0s],		train_loss: 2.0103,	val_loss: 1.3918
2:	[0s / 0s],		train_loss: 1.9269,	val_loss: 1.3689
3:	[0s / 0s],		train_loss: 1.8973,	val_loss: 1.3449
4:	[0s / 0s],		train_loss: 1.8519,	val_loss: 1.3293
5:	[0s / 0s],		train_loss: 1.8161,	val_loss: 1.3197
6:	[0s / 0s],		train_loss: 1.7956,	val_loss: 1.3174
7:	[0s / 0s],		train_loss: 1.7897,	val_loss: 1.3048
8:	[0s / 0s],		train_loss: 1.7569,	val_loss: 1.2910
9:	[0s / 0s],		train_loss: 1.7253,	val_loss: 1.2821
10:	[0s / 0s],		train_loss: 1.6970,	val_loss: 1.2747
11:	[0s / 0s],		train_loss: 1.6723,	val_loss: 1.2693
12:	[0s / 0s],		train_loss: 1.6564,	val_loss: 1.2664
13:	[0s / 0s],		train_loss: 1.6471,	val_loss: 1.2646
14:	[0s / 0s],		train_loss: 1.6422,	val_loss: 1.2640
15:	[0s / 0s],		train_loss: 1.6409,	val_loss: 1.2566
16:	[0s / 0s],		train_loss: 1.6136,	val_loss: 1.2492
17:	[0s / 0s],		train_loss: 1.5896,	val_loss: 1.2422
18:	[0s / 0s],		train_loss: 1.5661,	v

69:	[0s / 6s],		train_loss: 0.8976,	val_loss: 0.7946
70:	[0s / 6s],		train_loss: 0.8966,	val_loss: 0.7941
71:	[0s / 6s],		train_loss: 0.8956,	val_loss: 0.7937
72:	[0s / 6s],		train_loss: 0.8947,	val_loss: 0.7934
73:	[0s / 6s],		train_loss: 0.8938,	val_loss: 0.7931
74:	[0s / 6s],		train_loss: 0.8929,	val_loss: 0.7928
75:	[0s / 6s],		train_loss: 0.8919,	val_loss: 0.7925
76:	[0s / 6s],		train_loss: 0.8910,	val_loss: 0.7922
77:	[0s / 6s],		train_loss: 0.8901,	val_loss: 0.7920
78:	[0s / 6s],		train_loss: 0.8892,	val_loss: 0.7918
79:	[0s / 6s],		train_loss: 0.8882,	val_loss: 0.7916
80:	[0s / 6s],		train_loss: 0.8873,	val_loss: 0.7915
81:	[0s / 6s],		train_loss: 0.8864,	val_loss: 0.7914
82:	[0s / 6s],		train_loss: 0.8855,	val_loss: 0.7913
83:	[0s / 7s],		train_loss: 0.8846,	val_loss: 0.7913
84:	[0s / 7s],		train_loss: 0.8837,	val_loss: 0.7913
85:	[0s / 7s],		train_loss: 0.8827,	val_loss: 0.7913
86:	[0s / 7s],		train_loss: 0.8818,	val_loss: 0.7913
87:	[0s / 7s],		train_loss: 0.8809,	val_loss: 

28:	[0s / 1s],		train_loss: 1.3204,	val_loss: 1.1021
29:	[0s / 1s],		train_loss: 1.3153,	val_loss: 1.1009
30:	[0s / 1s],		train_loss: 1.3134,	val_loss: 1.1006
31:	[0s / 1s],		train_loss: 1.3129,	val_loss: 1.0750
32:	[0s / 1s],		train_loss: 1.2639,	val_loss: 1.0509
33:	[0s / 2s],		train_loss: 1.2115,	val_loss: 1.0288
34:	[0s / 2s],		train_loss: 1.1595,	val_loss: 1.0087
35:	[0s / 2s],		train_loss: 1.1113,	val_loss: 0.9903
36:	[0s / 2s],		train_loss: 1.0686,	val_loss: 0.9766
37:	[0s / 2s],		train_loss: 1.0323,	val_loss: 0.9663
38:	[0s / 2s],		train_loss: 1.0018,	val_loss: 0.9561
39:	[0s / 2s],		train_loss: 0.9759,	val_loss: 0.9475
40:	[0s / 2s],		train_loss: 0.9569,	val_loss: 0.9395
41:	[0s / 2s],		train_loss: 0.9446,	val_loss: 0.9331
42:	[0s / 2s],		train_loss: 0.9360,	val_loss: 0.9274
43:	[0s / 2s],		train_loss: 0.9309,	val_loss: 0.9217
44:	[0s / 2s],		train_loss: 0.9283,	val_loss: 0.9162
45:	[0s / 2s],		train_loss: 0.9272,	val_loss: 0.9112
46:	[0s / 2s],		train_loss: 0.9259,	val_loss: 

64:	[0s / 3s],		train_loss: 0.8937,	val_loss: 0.7963
65:	[0s / 3s],		train_loss: 0.8913,	val_loss: 0.7959
66:	[0s / 3s],		train_loss: 0.8890,	val_loss: 0.7955
67:	[0s / 3s],		train_loss: 0.8868,	val_loss: 0.7951
68:	[0s / 4s],		train_loss: 0.8846,	val_loss: 0.7948
69:	[0s / 4s],		train_loss: 0.8825,	val_loss: 0.7946
70:	[0s / 4s],		train_loss: 0.8804,	val_loss: 0.7944
71:	[0s / 4s],		train_loss: 0.8784,	val_loss: 0.7943
72:	[0s / 4s],		train_loss: 0.8764,	val_loss: 0.7944
73:	[0s / 4s],		train_loss: 0.8745,	val_loss: 0.7945
74:	[0s / 4s],		train_loss: 0.8726,	val_loss: 0.7948
8
0:	[0s / 0s],		train_loss: 1.2301,	val_loss: 1.0353
1:	[0s / 0s],		train_loss: 1.1308,	val_loss: 0.9724
2:	[0s / 0s],		train_loss: 1.0719,	val_loss: 0.9467
3:	[0s / 0s],		train_loss: 1.0508,	val_loss: 0.9088
4:	[0s / 0s],		train_loss: 1.0269,	val_loss: 0.8836
5:	[0s / 0s],		train_loss: 1.0109,	val_loss: 0.8713
6:	[0s / 0s],		train_loss: 1.0030,	val_loss: 0.8682
7:	[0s / 0s],		train_loss: 1.0007,	val_loss: 0.8537

53:	[0s / 2s],		train_loss: 1.0429,	val_loss: 0.9362
54:	[0s / 2s],		train_loss: 1.0402,	val_loss: 0.9342
55:	[0s / 2s],		train_loss: 1.0381,	val_loss: 0.9327
56:	[0s / 2s],		train_loss: 1.0363,	val_loss: 0.9315
57:	[0s / 2s],		train_loss: 1.0350,	val_loss: 0.9307
58:	[0s / 3s],		train_loss: 1.0340,	val_loss: 0.9300
59:	[0s / 3s],		train_loss: 1.0333,	val_loss: 0.9296
60:	[0s / 3s],		train_loss: 1.0329,	val_loss: 0.9294
61:	[0s / 3s],		train_loss: 1.0326,	val_loss: 0.9293
62:	[0s / 3s],		train_loss: 1.0325,	val_loss: 0.9292
63:	[0s / 3s],		train_loss: 1.0325,	val_loss: 0.9204
64:	[0s / 3s],		train_loss: 1.0235,	val_loss: 0.9111
65:	[0s / 3s],		train_loss: 1.0144,	val_loss: 0.9014
66:	[0s / 3s],		train_loss: 1.0052,	val_loss: 0.8916
67:	[0s / 3s],		train_loss: 0.9964,	val_loss: 0.8817
68:	[0s / 3s],		train_loss: 0.9888,	val_loss: 0.8719
69:	[0s / 3s],		train_loss: 0.9822,	val_loss: 0.8625
70:	[0s / 3s],		train_loss: 0.9756,	val_loss: 0.8533
71:	[0s / 3s],		train_loss: 0.9696,	val_loss: 

...

21:	[0s / 2s],		train_loss: 1.4726,	val_loss: 0.8497
22:	[0s / 2s],		train_loss: 1.4533,	val_loss: 0.8365
23:	[0s / 2s],		train_loss: 1.4334,	val_loss: 0.8258
24:	[0s / 2s],		train_loss: 1.4159,	val_loss: 0.8175
25:	[0s / 2s],		train_loss: 1.4016,	val_loss: 0.8115
26:	[0s / 2s],		train_loss: 1.3904,	val_loss: 0.8074
27:	[0s / 2s],		train_loss: 1.3823,	val_loss: 0.8044
28:	[0s / 2s],		train_loss: 1.3770,	val_loss: 0.8027
29:	[0s / 2s],		train_loss: 1.3739,	val_loss: 0.8019
30:	[0s / 3s],		train_loss: 1.3709,	val_loss: 0.8017
31:	[0s / 3s],		train_loss: 1.3722,	val_loss: 0.7867
32:	[0s / 3s],		train_loss: 1.3405,	val_loss: 0.7750
33:	[0s / 3s],		train_loss: 1.3039,	val_loss: 0.7684
34:	[0s / 3s],		train_loss: 1.2662,	val_loss: 0.7661
35:	[0s / 3s],		train_loss: 1.2323,	val_loss: 0.7659
36:	[0s / 3s],		train_loss: 1.1964,	val_loss: 0.7702
37:	[0s / 3s],		train_loss: 1.1592,	val_loss: 0.7747
38:	[0s / 3s],		train_loss: 1.1223,	val_loss: 0.7781
87
0:	[0s / 0s],		train_loss: 1.5288,	val_loss

59:	[0s / 3s],		train_loss: 0.8450,	val_loss: 0.8509
60:	[0s / 3s],		train_loss: 0.8449,	val_loss: 0.8509
61:	[0s / 3s],		train_loss: 0.8449,	val_loss: 0.8509
62:	[0s / 3s],		train_loss: 0.8449,	val_loss: 0.8509
63:	[0s / 3s],		train_loss: 0.8449,	val_loss: 0.8503
64:	[0s / 3s],		train_loss: 0.8429,	val_loss: 0.8497
65:	[0s / 3s],		train_loss: 0.8409,	val_loss: 0.8492
66:	[0s / 3s],		train_loss: 0.8390,	val_loss: 0.8488
67:	[0s / 3s],		train_loss: 0.8372,	val_loss: 0.8484
68:	[0s / 3s],		train_loss: 0.8353,	val_loss: 0.8480
69:	[0s / 3s],		train_loss: 0.8335,	val_loss: 0.8477
70:	[0s / 3s],		train_loss: 0.8318,	val_loss: 0.8474
71:	[0s / 4s],		train_loss: 0.8300,	val_loss: 0.8471
72:	[0s / 4s],		train_loss: 0.8284,	val_loss: 0.8469
73:	[0s / 4s],		train_loss: 0.8267,	val_loss: 0.8468
74:	[0s / 4s],		train_loss: 0.8251,	val_loss: 0.8467
75:	[0s / 4s],		train_loss: 0.8235,	val_loss: 0.8466
76:	[0s / 4s],		train_loss: 0.8220,	val_loss: 0.8465
77:	[0s / 4s],		train_loss: 0.8205,	val_loss: 

25:	[0s / 2s],		train_loss: 0.9456,	val_loss: 0.8138
26:	[0s / 2s],		train_loss: 0.9445,	val_loss: 0.8128
27:	[0s / 3s],		train_loss: 0.9437,	val_loss: 0.8122
28:	[0s / 3s],		train_loss: 0.9432,	val_loss: 0.8118
29:	[0s / 3s],		train_loss: 0.9429,	val_loss: 0.8117
30:	[0s / 3s],		train_loss: 0.9428,	val_loss: 0.8117
31:	[0s / 3s],		train_loss: 0.9427,	val_loss: 0.8086
32:	[0s / 3s],		train_loss: 0.9400,	val_loss: 0.8058
33:	[0s / 3s],		train_loss: 0.9375,	val_loss: 0.8028
34:	[0s / 3s],		train_loss: 0.9352,	val_loss: 0.8000
35:	[0s / 3s],		train_loss: 0.9331,	val_loss: 0.7975
36:	[0s / 3s],		train_loss: 0.9313,	val_loss: 0.7954
37:	[0s / 3s],		train_loss: 0.9298,	val_loss: 0.7933
38:	[0s / 3s],		train_loss: 0.9284,	val_loss: 0.7912
39:	[0s / 3s],		train_loss: 0.9271,	val_loss: 0.7892
40:	[0s / 3s],		train_loss: 0.9260,	val_loss: 0.7874
41:	[0s / 3s],		train_loss: 0.9249,	val_loss: 0.7858
42:	[0s / 3s],		train_loss: 0.9240,	val_loss: 0.7845
43:	[0s / 3s],		train_loss: 0.9232,	val_loss: 

17:	[0s / 1s],		train_loss: 1.3064,	val_loss: 0.9855
18:	[0s / 1s],		train_loss: 1.2746,	val_loss: 0.9696
19:	[0s / 1s],		train_loss: 1.2480,	val_loss: 0.9562
20:	[0s / 1s],		train_loss: 1.2213,	val_loss: 0.9468
21:	[0s / 1s],		train_loss: 1.1972,	val_loss: 0.9398
22:	[0s / 1s],		train_loss: 1.1756,	val_loss: 0.9339
23:	[0s / 1s],		train_loss: 1.1570,	val_loss: 0.9289
24:	[0s / 1s],		train_loss: 1.1411,	val_loss: 0.9251
25:	[0s / 1s],		train_loss: 1.1282,	val_loss: 0.9223
26:	[0s / 1s],		train_loss: 1.1188,	val_loss: 0.9204
27:	[0s / 1s],		train_loss: 1.1114,	val_loss: 0.9192
28:	[0s / 2s],		train_loss: 1.1072,	val_loss: 0.9185
29:	[0s / 2s],		train_loss: 1.1047,	val_loss: 0.9182
30:	[0s / 2s],		train_loss: 1.1032,	val_loss: 0.9181
31:	[0s / 2s],		train_loss: 1.1031,	val_loss: 0.9122
32:	[0s / 2s],		train_loss: 1.0791,	val_loss: 0.9068
33:	[0s / 2s],		train_loss: 1.0570,	val_loss: 0.9017
34:	[0s / 2s],		train_loss: 1.0376,	val_loss: 0.8967
35:	[0s / 2s],		train_loss: 1.0198,	val_loss: 

47:	[0s / 3s],		train_loss: 1.0248,	val_loss: 0.7692
48:	[0s / 3s],		train_loss: 1.0187,	val_loss: 0.7691
49:	[0s / 3s],		train_loss: 1.0129,	val_loss: 0.7691
50:	[0s / 3s],		train_loss: 1.0079,	val_loss: 0.7690
51:	[0s / 3s],		train_loss: 1.0037,	val_loss: 0.7690
52:	[0s / 3s],		train_loss: 1.0002,	val_loss: 0.7690
53:	[0s / 3s],		train_loss: 0.9971,	val_loss: 0.7691
54:	[0s / 3s],		train_loss: 0.9947,	val_loss: 0.7691
98
0:	[0s / 0s],		train_loss: 1.8903,	val_loss: 1.3787
1:	[0s / 0s],		train_loss: 1.7667,	val_loss: 1.3310
2:	[0s / 0s],		train_loss: 1.6896,	val_loss: 1.3096
3:	[0s / 0s],		train_loss: 1.6544,	val_loss: 1.2762
4:	[0s / 0s],		train_loss: 1.5968,	val_loss: 1.2445
5:	[0s / 0s],		train_loss: 1.5480,	val_loss: 1.2236
6:	[0s / 0s],		train_loss: 1.5136,	val_loss: 1.2176
7:	[0s / 0s],		train_loss: 1.5049,	val_loss: 1.1839
8:	[0s / 0s],		train_loss: 1.4471,	val_loss: 1.1520
9:	[0s / 0s],		train_loss: 1.3869,	val_loss: 1.1247
10:	[0s / 0s],		train_loss: 1.3339,	val_loss: 1.1041
